In [1]:
//Assignment 1
//By: Michael Tahirovic, Ben Hamilton, Zach Pereira
//Emails: tahi9525@mylaurier.ca, hami5217@mylaurier.ca, pere
//Date: 2024-09-26
//------------------------------------------------------------------------------------------
//Question 1:
//Write a C/C++ program in which a parent forks a child, and the child forks a grandchild.
//The grandchild writes: “My process id is X, my parent’s id is Y , and my Grandparent’s id is Z” (where
//X, Y , and Z are the process ids). The program should produce no other output.
//------------------------------------------------------------------------------------------
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
int main() {
pid_t pid1, pid2;
// Parent forks a child
pid1 = fork();
if (pid1 < 0) {
    // Fork failed
    (stderr, "Fork failed\n");
    exit(1);
} else if (pid1 == 0) {
    // Inside child process
    pid_t grandparent_id = getppid(); // Get the grandparent (parent's) ID
    // Child forks the grandchild
    pid2 = fork();
    if (pid2 < 0) {
    // Fork failed
    fprintf(stderr, "Fork failed\n");
    exit(1);
    } else if (pid2 == 0) {
        // Inside grandchild process
        printf("My process id is %d, my parent’s id is %d, and my grandparent’s id is %d\n",
        getpid(), getppid(), grandparent_id);
        exit(0); // Grandchild exits after printing
        } else {
            // Inside child process
            wait(NULL); // Wait for grandchild to finish
            exit(0); // Child exits after forking grandchild
            }
    } else {
    // Inside parent process
    wait(NULL); // Wait for child to finish
    exit(0); // Parent exits
    }
    return 0;
}

My process id is 15321, my parent’s id is 15320, and my grandparent’s id is 15317


In [2]:
//Assignment 1
//By: Michael Tahirovic, Ben Hamilton, Zach Pereira
//Emails: tahi9525@mylaurier.ca, hami5217@mylaurier.ca, pere
//Date: 2024-09-26
//------------------------------------------------------------------------------------------
//Question 2a:
//Suppose we have n programs p1,...,pn to be executed. The programs are described in an input text file. 
//(a) Suppose the programs are independent from each other, so that they can be executed in
//parallel. Write a parallel program executor, receiving the absolute path of the
//abovementioned input file as a command-line argument. This program runs all the programs
//listed in the file in parallel.
//------------------------------------------------------------------------------------------
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <string.h>
int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <input_file>\n", argv[0]);
        exit(1);
    }
    // Open input file
    FILE *input_file = fopen(argv[1], "r");
    if (input_file == NULL) {
        perror("Error opening file");
        exit(1);
    }
    // Read number of programs
    int n;
    if (fscanf(input_file, "%d", &n) != 1) {
        fprintf(stderr, "Error reading the number of programs\n");
        fclose(input_file);
        exit(1);
    }
    // Dynamically allocate memory for program paths
    char **paths = malloc(n * sizeof(char *));
    if (paths == NULL) {
        perror("Memory allocation failed");
        fclose(input_file);
        exit(1);
    }
    // Read program paths
    for (int i = 0; i < n; i++) {
        paths[i] = malloc(256 * sizeof(char)); // Allocating memory for each path
        if (paths[i] == NULL || fscanf(input_file, "%255s", paths[i]) != 1) {
            fprintf(stderr, "Error reading program path\n");
            fclose(input_file);
            for (int j = 0; j <= i; j++) free(paths[j]); // Free previously allocated memory
            free(paths);
            exit(1);
        }
    }
    fclose(input_file);
    // Fork child processes for each program to run in parallel
    for (int i = 0; i < n; i++) {
        pid_t pid = fork();
        if (pid < 0) {
            // Fork failed
            perror("Fork failed");
            exit(1);
        } else if (pid == 0) {
            // Child process: Execute the program
            char *args[] = {paths[i], NULL};
            execvp(args[0], args);
            // If execvp fails
            perror("Error executing program");
            exit(1);
        }
    }
    // Parent process: Wait for all children to finish
    for (int i = 0; i < n; i++) {
        wait(NULL);
    }
    // Free allocated memory
    for (int i = 0; i < n; i++) {
        (paths[i]);
    }
    free(paths);
    printf("All programs executed in parallel.\n");
    return 0;
}

Usage: /tmp/tmpzov862dr.out <input_file>
[C kernel] Executable exited with code 1

In [3]:
//Assignment 1
//By: Michael Tahirovic, Ben Hamilton, Zach Pereira
//Emails: tahi9525@mylaurier.ca, hami5217@mylaurier.ca, pere
//Date: 2024-09-26
//------------------------------------------------------------------------------------------
//Question 2a:
//Suppose we have n programs p1,...,pn to be executed. The programs are described in an input text file. 
//(b) Now suppose each program pi depends on all the programs p1,...,pi−1. This means the programs
//should be executed sequentially, first completing p1, then p2 and so on. Modify the parallel
//program executor you designed in part (a) to obtain a sequential program executor.
//------------------------------------------------------------------------------------------
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <string.h>
int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <input_file>\n", argv[0]);
        exit(1);
    }
    // Open input file
    FILE *input_file = fopen(argv[1], "r");
    if (input_file == NULL) {
        perror("Error opening file");
        exit(1);
    }
    // Read number of programs
    int n;
    if (fscanf(input_file, "%d", &n) != 1) {
        fprintf(stderr, "Error reading the number of programs\n");
        fclose(input_file);
        exit(1);
    }
    // Dynamically allocate memory for program paths
    char **paths = malloc(n * sizeof(char *));
    if (paths == NULL) {
        perror("Memory allocation failed");
        fclose(input_file);
        exit(1);
    }
    // Read program paths
    for (int i = 0; i < n; i++) {
        paths[i] = malloc(256 * sizeof(char)); // Allocating memory for each path
        if (paths[i] == NULL || fscanf(input_file, "%255s", paths[i]) != 1) {
            fprintf(stderr, "Error reading program path\n");
            fclose(input_file);
            for (int j = 0; j <= i; j++) free(paths[j]); // Free previously allocated memory
                free(paths);
            exit(1);
        }
    }
    fclose(input_file);
    // Execute programs sequentially
    for (int i = 0; i < n; i++) {
        pid_t pid = fork();
        if (pid < 0) {
            // Fork failed
            perror("Fork failed");
            exit(1);
        } else if (pid == 0) {
            // Child process: Execute program
            char *args[] = {paths[i], NULL};
            execvp(args[0], args);
            // If execvp fails
            perror("Error executing program");
            exit(1);
        } else {
            // Parent process: Wait for the current child to finish
            wait(NULL);
        }
    }
    // Free allocated memory
    for (int i = 0; i < n; i++) {
        free(paths[i]);
    }
    free(paths);
    printf("All programs executed sequentially.\n");
    return 0;
}

Usage: /tmp/tmpnrgifz44.out <input_file>
[C kernel] Executable exited with code 1

In [ ]:
//Assignment 1
//By: Michael Tahirovic, Ben Hamilton, Zach Pereira
//Emails: tahi9525@mylaurier.ca, hami5217@mylaurier.ca, pere
//Date: 2024-09-26
//------------------------------------------------------------------------------------------
//Question 3:
//Write a C/C++ program that replicates the functionality of the Unix pipeline command
//$ ps xao sid | sort -u | wc -l
//This command counts the number of unique session IDs currently active on the system. Your
//implementation should focus on process creation and file descriptor management. Create three
//separate processes corresponding to each part of the pipeline (ps, sort, and wc). Each process
//should execute the respective command as a new program. Your primary task is to ensure proper
//control and redirection of file descriptors to mimic the behavior of the pipe (|) operator in a Unix
//shell. Remember, the goal is not to reimplement the commands themselves but to orchestrate their
//execution and intercommunication in a C/C++ program.
//------------------------------------------------------------------------------------------
